# Which Economic Tasks are Performed with AI? Evidence from Millions of Claude Conversations

_Handa et al., 2025_

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from textwrap import wrap
import numpy as np

palette = sns.color_palette("colorblind")

### Create O*NET / SOC Merged Dataframe

In [2]:
def merge_onet_soc_data() -> pd.DataFrame:
    """
    Merges O*NET task statements with SOC (Standard Occupational Classification) data
    based on major group codes.
    
    Args:
        onet_path (str): Path to the O*NET task statements CSV file
        soc_path (str): Path to the SOC structure CSV file
    
    Returns:
        pd.DataFrame: Merged DataFrame containing O*NET data with SOC major group titles
    """

    # Read and process O*NET data
    onet_df = pd.read_csv("onet_task_statements.csv")
    onet_df["soc_major_group"] = onet_df["O*NET-SOC Code"].str[:2]
    
    # Read and process SOC data
    soc_df = pd.read_csv("SOC_Structure.csv")
    soc_df = soc_df.dropna(subset=['Major Group'])
    soc_df["soc_major_group"] = soc_df["Major Group"].str[:2]
    
    # Merge datasets
    merged_df = onet_df.merge(
        soc_df[['soc_major_group', 'SOC or O*NET-SOC 2019 Title']],
        on='soc_major_group',
        how='left'
    )

    return merged_df

In [3]:
task_occupations_df = merge_onet_soc_data()

In [ ]:
task_occupations_df["Title"].nunique()

In [ ]:
# Update cluster mappings to include data from the merged_df
task_occupations_df["task_normalized"] = task_occupations_df["Task"].str.lower().str.strip()
# Some tasks are included multiple times, so we need to count the number of occurrences per task
task_occupations_df["n_occurrences"] = task_occupations_df.groupby("task_normalized")["Title"].transform("nunique")

task_occupations_df

### Load Task Mappings and Join

In [6]:
task_mappings_df = pd.read_csv("onet_task_mappings.csv")

In [ ]:
grouped_with_occupations = task_mappings_df.merge(
    task_occupations_df,
    left_on="task_name",
    right_on="task_normalized",
    how="left"
)

grouped_with_occupations["pct_occ_scaled"] = 100 * (grouped_with_occupations["pct"] / grouped_with_occupations["n_occurrences"]) / (grouped_with_occupations["pct"] / grouped_with_occupations["n_occurrences"]).sum()
grouped_with_occupations["pct_occ_scaled"].sum()

In [ ]:
grouped_with_occupations

## EXPERIMENTS

### TASKS

In [9]:
# Set style and increase font sizes
plt.rcParams['font.size'] = 12  # Base font size
plt.rcParams['axes.titlesize'] = 14  # Title font size
plt.rcParams['axes.labelsize'] = 12  # Axis labels size
plt.rcParams['xtick.labelsize'] = 11  # X-axis tick labels size
plt.rcParams['ytick.labelsize'] = 11  # Y-axis tick labels size
plt.rcParams['legend.fontsize'] = 11  # Legend font size
plt.rcParams['figure.titlesize'] = 16  # Figure title size

# If you're using seaborn, you can also set its context
sns.set_context("notebook", font_scale=1.2)

In [ ]:
# Get top 10 tasks overall to ensure consistent comparison
total_tasks = (grouped_with_occupations.groupby("Task")["pct_occ_scaled"]
               .sum()
               .sort_values(ascending=False))
top_10_tasks = total_tasks.head(10).index

# Create plot dataframe with all groups
plot_df = (grouped_with_occupations[grouped_with_occupations["Task"].isin(top_10_tasks)]
           .groupby(["Task"])["pct_occ_scaled"]
           .sum()
           .reset_index())

# Sort tasks by overall frequency
task_order = (plot_df.groupby("Task")["pct_occ_scaled"]
              .sum()
              .sort_values(ascending=False)
              .index)
plot_df["Task"] = pd.Categorical(plot_df["Task"], categories=task_order, ordered=True)

# Create the plot
plt.figure(figsize=(16, 12))
sns.barplot(
    data=plot_df,
    x="pct_occ_scaled",
    y="Task",
    color=palette[0],
)

# Wrap task titles
ax = plt.gca()
ax.set_yticklabels(['\n'.join(wrap(label.get_text(), width=40)) 
                    for label in ax.get_yticklabels()])

# Modify legend
handles, labels = ax.get_legend_handles_labels()

# Wrap task labels
ax = plt.gca()
ax.set_yticklabels(['\n'.join(wrap(label.get_text(), width=40)) 
                    for label in ax.get_yticklabels()])

# Format x-axis as percentages
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.1f}%'))

# Customize the plot
plt.title('Top Tasks by % of Conversations')
plt.xlabel('Percentage of Records')
plt.ylabel('O*NET Task')

# Adjust layout to prevent label cutoff
plt.tight_layout()

plt.show()


### OCCUPATIONS

In [ ]:
grouped_with_occupations.groupby("Title")["pct_occ_scaled"].sum()

In [ ]:
# Calculate percentages per group and occupation
plot_df = (grouped_with_occupations.groupby("Title")["pct_occ_scaled"]
           .sum()
           .reset_index())

# Get top occupations overall
total_occs = (plot_df.groupby("Title")["pct_occ_scaled"]
              .sum()
              .sort_values(ascending=False))
top_occs = total_occs.head(15).index

# Filter for top occupations
plot_df = plot_df[plot_df["Title"].isin(top_occs)]

# Sort occupations by overall frequency
occ_order = (plot_df.groupby("Title")["pct_occ_scaled"]
             .sum()
             .sort_values(ascending=False)
             .index)
plot_df["Title"] = pd.Categorical(plot_df["Title"], categories=occ_order, ordered=True)

# Create the plot
plt.figure(figsize=(18, 16))
sns.barplot(
    data=plot_df,
    x="pct_occ_scaled",
    y="Title",
    color=palette[0],
)

# Wrap occupation titles
ax = plt.gca()
ax.set_yticklabels(['\n'.join(wrap(label.get_text(), width=40)) 
                    for label in ax.get_yticklabels()])

# Format x-axis as percentages
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.1f}%'))

# Customize the plot
plt.title('Top Occupations by % of Conversations')
plt.xlabel('Percentage of Conversations')
plt.ylabel('Occupation')

# Adjust layout to prevent label cutoff
plt.tight_layout()

plt.show()

### OCCUPATIONAL CATEGORIES

In [ ]:
# Calculate percentages per group and occupational category
plot_df = (grouped_with_occupations.groupby("SOC or O*NET-SOC 2019 Title")["pct_occ_scaled"]
           .sum()
           .reset_index())

# Sort categories by group-1 frequency
cat_order = plot_df.sort_values("pct_occ_scaled", ascending=False)["SOC or O*NET-SOC 2019 Title"]
plot_df["SOC or O*NET-SOC 2019 Title"] = pd.Categorical(
    plot_df["SOC or O*NET-SOC 2019 Title"], 
    categories=cat_order, 
    ordered=True
)

# Create the plot
plt.figure(figsize=(18, 16))
sns.barplot(
    data=plot_df,
    x="pct_occ_scaled",
    y="SOC or O*NET-SOC 2019 Title",
    color=palette[0],
)

# Wrap category labels and remove " Occupations" string
ax = plt.gca()
ax.set_yticklabels(['\n'.join(wrap(label.get_text().replace(" Occupations", ""), width=60)) 
                    for label in ax.get_yticklabels()])

# Format x-axis as percentages
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.1f}%'))

# Customize the plot
plt.title('Occupational Categories by % of Conversations')
plt.xlabel('Percentage of Conversations')
plt.ylabel('Occupational Category')

# Adjust layout to prevent label cutoff
plt.tight_layout()

plt.show()

In [ ]:
grouped_with_occupations

In [15]:
# Load employment data
bls_employment_df = pd.read_csv("bls_employment_may_2023.csv")

claude_employment_df = grouped_with_occupations.groupby("SOC or O*NET-SOC 2019 Title")["pct_occ_scaled"].sum().reset_index(name='claude_distribution')

employment_df = claude_employment_df.merge(bls_employment_df, 
                                     on='SOC or O*NET-SOC 2019 Title',
                                     how='left')

In [ ]:
# Calculate percentages and setup data
plot_df = employment_df.copy()

def get_distribution(df, value_column):
   total = df[value_column].sum()
   return (df[value_column] / total * 100).round(1)

plot_df['bls_pct'] = get_distribution(plot_df, 'bls_distribution')
plot_df['claude_pct'] = get_distribution(plot_df, 'claude_distribution')
plot_df['clean_label'] = plot_df['SOC or O*NET-SOC 2019 Title'].str.replace(' Occupations', '')
plot_df['pct_difference'] = plot_df['claude_pct'] - plot_df['bls_pct']
plot_df = plot_df.sort_values('bls_pct', ascending=True)

# Create the plot
fig, ax = plt.subplots(figsize=(20, 12))

# Set colors
claude_color = palette[1] 
bls_color = palette[0] 

# Create lines and circles
y_positions = range(len(plot_df))
for i, row in enumerate(plot_df.itertuples()):
   # Determine color based on which value is larger
   line_color = claude_color if row.claude_pct > row.bls_pct else bls_color
   
   # Draw the line between bls and claude percentages
   ax.plot([row.bls_pct, row.claude_pct], [i, i], 
           color=line_color, 
           linestyle='-', 
           linewidth=2.5,
           zorder=1)
   
   # Determine label positioning
   if row.claude_pct > row.bls_pct:
       bls_ha = 'right'
       claude_ha = 'left'
       bls_offset = -0.4
       claude_offset = 0.4
   else:
       bls_ha = 'left'
       claude_ha = 'right'
       bls_offset = 0.4
       claude_offset = -0.4

   # Plot BLS percentage
   ax.scatter([row.bls_pct], [i], 
              color=bls_color,
              s=200,
              zorder=2,
              label='% of U.S. workers (BLS)' if i == 0 else "")
   ax.text(row.bls_pct + bls_offset,
           i,
           f'{row.bls_pct:.1f}%',
           ha=bls_ha,
           va='center',
           color=bls_color)
   
   # Plot Claude's percentage
   ax.scatter([row.claude_pct], [i], 
              color=claude_color,
              s=200,
              zorder=2,
              label='% of Claude conversations' if i == 0 else "")
   ax.text(row.claude_pct + claude_offset,
           i,
           f'{row.claude_pct:.1f}%',
           ha=claude_ha,
           va='center',
           color=claude_color)

# Customize the plot
ax.set_xlabel('Percentage')
ax.set_ylabel('Occupational Category')

# Add percentage formatter to x-axis
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.1f}%'))

# Set y-axis labels
ax.set_yticks(y_positions)
ax.set_yticklabels(plot_df['clean_label'])

# Add legend
handles, labels = ax.get_legend_handles_labels()
handles = handles[::-1]
labels = labels[::-1]
ax.legend(handles, labels, loc='lower right', bbox_to_anchor=(1.0, 0.0))

# Adjust grid and layout
ax.grid(axis='x', linestyle='--', alpha=0.3)
ax.set_axisbelow(True)

# Set axis limits with padding
max_val = max(plot_df['bls_pct'].max(), plot_df['claude_pct'].max())
min_val = min(plot_df['bls_pct'].min(), plot_df['claude_pct'].min())
padding = (max_val - min_val) * 0.15
ax.set_xlim(min_val - padding, max_val + padding)
ax.set_ylim(-1, len(plot_df))

# Adjust layout
plt.tight_layout()
plt.show()

### USAGE BY WAGE

In [17]:
# Read and process wage data
wage_df = pd.read_csv("wage_data.csv")

In [ ]:
wage_df

In [ ]:
# Join wage and occupation data
grouped_with_occupations_and_wage = grouped_with_occupations.merge(wage_df, left_on="O*NET-SOC Code", right_on="SOCcode", how="left")
grouped_with_occupations_and_wage

In [ ]:
def create_wage_distribution_plot(plot_df):
    # Create figure
    plt.figure(figsize=(24, 12))
    
    # Create scatter plot
    sns.scatterplot(data=plot_df,
                    x='MedianSalary',
                    y='pct_occ_scaled',
                    alpha=0.5,
                    size='pct_occ_scaled',
                    sizes=(60, 400),
                    color=palette[0],
                    legend=False)
    
    # Style the plot
    plt.xlabel('Median Wage ($)')
    plt.ylabel('Percent of Conversations')
    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.1f}%'.format(y)))
    
    # Add title
    plt.title('Wage Distribution by % of Conversations'), 
    
    # Annotate points
    # Top points by percentage
    top_n = 7
    for _, row in plot_df.nlargest(top_n, 'pct_occ_scaled').iterrows():
        plt.annotate('\n'.join(wrap(row['Title'], width=20)), 
                    (row['MedianSalary'], row['pct_occ_scaled']),
                    xytext=(5, 5), 
                    textcoords='offset points')
    
    # Extreme salary points
    n_extremes = 2
    # Annotate lowest and highest salaries
    for df_subset in [plot_df.nsmallest(n_extremes, 'MedianSalary'),
                     plot_df.nlargest(n_extremes, 'MedianSalary')]:
        for i, row in enumerate(df_subset.iterrows()):
            if i != 0:  # Skip if already annotated in top_n
                plt.annotate('\n'.join(wrap(row[1]['Title'], width=20)), 
                            (row[1]['MedianSalary'], row[1]['pct_occ_scaled']),
                            xytext=(5, -15),
                            textcoords='offset points')
    
    # Formatting
    plt.ylim(bottom=0)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
    
    plt.tight_layout()
    
    plt.show()
    plt.close()

# Create aggregation dictionary, excluding groupby columns
groupby_cols = ["Title"]
agg_dict = {col: 'first' for col in grouped_with_occupations_and_wage.columns 
            if col not in groupby_cols}
agg_dict['pct_occ_scaled'] = 'sum'


plot_df = (grouped_with_occupations_and_wage
            .groupby(groupby_cols)
            .agg(agg_dict)
            .reset_index()
            .copy())
    
# Filter out null values and very low salaries
plot_df = plot_df[plot_df["MedianSalary"].notnull() & 
                    (plot_df["MedianSalary"] > 100)]
    
# Create and save plot for current group
create_wage_distribution_plot(plot_df)

### AUTOMATION VS AUGMENTATION

In [ ]:
automation_vs_augmentation_df = pd.read_csv("automation_vs_augmentation.csv")

def adjust_color_brightness(color, factor):
    """Adjust the brightness of a color by a factor"""
    # Convert color to RGB if it's not already
    if isinstance(color, str):
        color = mcolors.to_rgb(color)
    # Make brighter by scaling RGB values
    return tuple(min(1.0, c * factor) for c in color)

def plot_interaction_modes(df):
    # Load in dataframe
    plot_df = df.copy()
    
    # Convert cluster_name to lowercase first, then filter and normalize
    plot_df['interaction_type'] = plot_df['interaction_type'].str.lower()
    plot_df = plot_df[plot_df['interaction_type'] != 'none']
    total = plot_df['pct'].sum()
    plot_df['pct'] = plot_df['pct'] / total
    
    # Create category mapping
    category_map = {
        'directive': 'Automation',
        'feedback loop': 'Automation',
        'task iteration': 'Augmentation',
        'learning': 'Augmentation',
        'validation': 'Augmentation'
    }
    
    # Add category column
    plot_df['category'] = plot_df['interaction_type'].map(category_map)
    
    # Convert to title case for plotting
    plot_df['interaction_type'] = plot_df['interaction_type'].str.title()
    
    # Create color variants
    colors_a = [
        palette[1],
        adjust_color_brightness(palette[1], 1.3)
    ]
    
    colors_b = [
        palette[2],
        adjust_color_brightness(palette[2], 1.3),
        adjust_color_brightness(palette[2], 1.6)
    ]
    
    # Create the stacked bar plot
    plt.figure(figsize=(16, 6))
    
    # Create separate dataframes for each category and sort them to match visual order
    automation_df = plot_df[plot_df['category'] == 'Automation'].sort_values('interaction_type', ascending=False)
    augmentation_df = plot_df[plot_df['category'] == 'Augmentation'].sort_values('interaction_type', ascending=False)
    
    # Calculate positions for the bars
    bar_positions = [0, 1]
    bar_width = 0.8
    
    # Create the stacked bars for each category
    left_auto = 0
    handles, labels = [], []  # Initialize empty lists for legend ordering
    
    # First plot automation bars but save their handles/labels
    auto_handles, auto_labels = [], []
    for i, (_, row) in enumerate(automation_df.iterrows()):
        bar = plt.barh(0, row['pct'], left=left_auto, height=bar_width, 
                color=colors_a[i])
        auto_handles.append(bar)
        auto_labels.append(row['interaction_type'])
        plt.text(left_auto + row['pct']/2, 0, 
                f'{row["pct"]*100:.1f}%', 
                ha='center', va='center',
                color='white')
        left_auto += row['pct']
    
    # Plot augmentation bars and save handles/labels
    left_aug = 0
    aug_handles, aug_labels = [], []
    for i, (_, row) in enumerate(augmentation_df.iterrows()):
        bar = plt.barh(1, row['pct'], left=left_aug, height=bar_width,
                color=colors_b[i])
        aug_handles.append(bar)
        aug_labels.append(row['interaction_type'])
        plt.text(left_aug + row['pct']/2, 1, 
                f'{row["pct"]*100:.1f}%', 
                ha='center', va='center',
                color='white')
        left_aug += row['pct']
    
    # Customize the plot
    plt.yticks(bar_positions, ['Automation', 'Augmentation'])
    plt.xlabel('Percentage of Conversations')
    
    # Create legend with custom order
    # Combine handles and labels in the desired order
    all_handles = aug_handles + auto_handles
    all_labels = aug_labels + auto_labels
    
    # Create legend with specified order
    desired_order = ['Validation', 'Task Iteration', 'Learning', 'Feedback Loop', 'Directive'] 
    ordered_handles = []
    ordered_labels = []
    
    for label in desired_order:
        idx = all_labels.index(label)
        ordered_handles.append(all_handles[idx])
        ordered_labels.append(all_labels[idx])
    
    plt.legend(ordered_handles, ordered_labels, loc='lower right')
    
    plt.tight_layout()

plot_interaction_modes(automation_vs_augmentation_df)